# A test of Variational Monte Carlo on some Toy Models

Here we only deal with a toy model based on a Jastrow solution of Fermi-Hubbard model. We first validate this method in 1D, then apply this to the 2D case. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

## Fermi-Hubbard model

$$ H = -t \sum_{\langle ij \rangle} \sum_{\sigma} c_{i, \sigma}^{\dagger} c_{j, \sigma} + U \sum_{i} n_{i\uparrow} n_{i\downarrow}$$

(For 2D, there are more N.N. sites)

periodic boundary condition

## Noninteracting state (choose U=0 here instead of Hartree-Fock)

$$ c_{i\uparrow} = d_i $$

$$ c_{i\downarrow} = d_{i+L} $$

index $I$ running from 1 to 2L


In [7]:
# system size (1D)
L = 4
N = 2
U = 2         # take tunneling strength as a unit
H1 = np.zeros((2*L,2*L))
for i in range(2*L-1):
    H1[i, i+1] = -1
H1[L-1, 0] = -1
H1[2*L-1, L] = -1
H1[L-1, L] = 0
H = H1 + H1.T
e, phi = np.linalg.eig(H)
phi_0 = phi[np.argsort(e)]
# print(e, np.argsort(e), phi_0)

## Jastrow-like Wavefunction

$$ |\psi \rangle = exp(-\frac{1}{2} v \sum_{i} n_{i\uparrow} n_{i\downarrow}) |\phi _0 \rangle $$

In [12]:
# initialization
v = np.random.randn()
x = np.random.choice(2*L, N, replace=False)
site_empty = np.delete(np.arange(2*L), x)
x_ext = x - L
site_double = np.intersect1d(x, x_ext)
# print(x, x_ext, site_double)


## Computation of expectations and variances of local operators $O$ (here for Hamiltonian $H$ especially)




In [ ]:
# energy and variance


## Update of determinant $|\frac{\phi_0(x^\prime)}{\phi_0(x)}|^2$ :

$x^\prime$ differs from x with only one index, x ranges from 0 to 2L-1 (spin up for 0 ~ L-1) 

### Some details: 

Notation:



## Update of Jastrow factor:

Since there are only diagnoal terms in Gutzwiller factor, we only need to keep track with doubly occupied sites, which are nonvanishing in the sum on the exponential term.


In [11]:
# randomly choose a new configuration
index_change_orbit = np.random.randint(N)


# update of Jastrow factor


# update of determinant



In [ ]:
# energy



In [ ]:
# variation


In [2]:
# test

array([0., 0.])